# Sentinel-2 image composite
Gemasakti Adzan

2021-09-23

In [ ]:
# packages
import ee 
import geopandas as gpd
# import geemap
import ipyleaflet
from ipywidgets import fixed, interact, interact_manual, widgets
from IPython.display import display, clear_output
from datetime import datetime
import math

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize()

In [ ]:
# Local data
adm2_loc = gpd.read_file('./data/adm2.shp')
adm2_list = adm2_loc['ADM2_EN'].tolist()

# GEE assets and data
adm1 = ee.FeatureCollection('projects/ee-jatim/assets/adm1') # batas provinsi Jawa Timur
adm2 = ee.FeatureCollection('projects/ee-jatim/assets/adm2') # batas kabupaten/kota di Jawa Timur
s2_coll = ee.ImageCollection('COPERNICUS/S2')

In [ ]:
# create some widgets
startDate_widget = widgets.DatePicker(description = 'Pick a Start Date', disable = False)
endDate_widget = widgets.DatePicker(description = 'Pick an End Date', disable = False)
region_widget = widgets.Dropdown(options = adm2_list, value = None, description = "Pick a kabupaten/kota", disabled = False)
exp_widget = widgets.Checkbox(value = False, description = 'Check to export image to Google Drive', disabled = False, indent = False)
folderExp_widget = widgets.Text(placeholder = 'Write folder name in Google Drive to save image', description = 'Folder name')

In [ ]:
# functions
# get sentinel data
def importData(region,startDate,endDate):

    startDate = startDate_widget.value.strftime('%Y') + "-" + startDate_widget.value.strftime('%m') + "-" + startDate_widget.value.strftime('%d')
    endDate = endDate_widget.value.strftime('%Y') + "-" + endDate_widget.value.strftime('%m') + "-" + endDate_widget.value.strftime('%d')
    region = adm2.filter(ee.Filter.eq('ADM2_EN',region_widget.value))

    s2 = s2_coll.filterDate(startDate,endDate).filterBounds(region)

    return s2

def QAMaskCloud(collection):
    
    def maskClouds(image):
        qa = image.select('QA60').int16()
        cloudBitMask = math.pow(2, 10)
        cirrusBitMask = math.pow(2, 11)
        mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
        return image.updateMask(mask)
    
    collection = collection.map(maskClouds)
    
    return collection

# if using ipyleaflet
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_fetcher = map_id['tile_fetcher']
  return tile_fetcher.url_format

In [ ]:
# Example using ipyleaflet
n = ipyleaflet.Map(center = (-7.27, 112.74), zoom = 10, scroll_wheel_zoom = True)
layer_control = ipyleaflet.LayersControl(position = 'topright')
n.add_control(layer_control)


def showMap(region,startDate,endDate,expTrigger,folderName):
    s2_image = importData(region, startDate, endDate)
    s2_image = QAMaskCloud(s2_image)
    s2_image = s2_image.median().clip(adm2.filter(ee.Filter.eq('ADM2_EN',region)))
    n.add_layer(ipyleaflet.TileLayer(url = GetTileLayerUrl(s2_image.visualize(min = 40, max = 4000, gamma = 1.1, bands = ['B8','B10','B11']))))
    x = adm2_loc[adm2_loc['ADM2_EN'] == region]
    x['geometry'] = x.representative_point()
    x['lat'] = x['geometry'].y
    x['lon'] = x['geometry'].x
    lat = x.iloc[0]['lat']
    lon = x.iloc[0]['lon']
    n.center = (lat,lon)
    task = ee.batch.Export.image.toDrive(**{
        'image': s2_image,
        'description': 'Sentinel2_' + str(startDate) + '_' + str(endDate) + '_' + region + '_' + 'py',
        'folder': folderName,
        'scale': 10,
        'region': adm2.filter(ee.Filter.eq('ADM2_EN',region)).geometry()})
    if expTrigger == True:
        task.start()
    else:
        pass

interact_manual(showMap, region = region_widget, startDate = startDate_widget, endDate = endDate_widget, expTrigger = exp_widget, folderName = folderExp_widget)
display(n)